In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from cvxopt import matrix, solvers
merged_data = pd.read_csv('../../data/merged_data_truncated.csv')

In [14]:
merged_data.shape

(17, 8)

In [15]:
merged_data.head()

,Open,High,Low,Close,Adj Close,Volume,Sentiment1,Sentiment2
0,173.800003,177.990005,173.179993,177.490005,177.490005,57224100,0.9480,0.096352
1,176.809998,179.050003,175.800003,178.990005,178.990005,42390800,-0.2204,0.071875
2,178.100006,179.720001,177.949997,178.389999,178.389999,43698000,0.9768,0.170134
3,178.199997,179.850006,177.600006,179.800003,179.800003,47551100,0.9854,0.143628
4,180.070007,182.339996,179.039993,180.710007,180.710007,56743100,0.9934,0.106639


In [16]:
# y_open=merged_data['Open']
# y_close=merged_data['Close']
y_data=merged_data.drop(columns=['High','Low','Adj Close','Volume','Sentiment1','Sentiment2','Close'])
# print(y_data)
# y_array=y_data.values
# print(y_array)
input_vector= merged_data.drop(columns=['Open','Close','Adj Close'])
# input_array = input_vector.values
size=len(input_vector)
print(size)
train_ratio=0.6
train_size=int(size*train_ratio)
    


# #Splitting the data in training and testing data 


#Training data
X_train=input_vector[:train_size]
# print(X_train)
y_train=y_data[:train_size]
    
#Testing data

X_test=input_vector[train_size:]
y_test=y_data[train_size:]

#Conversion from pandas data frame to numpy data frame
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values
# print(X_train)






17


In [17]:
def rbf_kernel(x1, x2, gamma='scale'):
      # If gamma is set to 'scale', calculate it based on the number of features in x1
    if gamma == 'scale':
        gamma = 1 / (2 * len(x1))
    #this is the radial basis kernel evaluation based on norm
    return np.exp(-gamma * np.linalg.norm(x1 - x2) ** 2)

In [232]:
X_train.shape
for i in X:
    print(i)

[0.03283154]
[0.03953824]
[0.04320011]
[0.05396743]
[0.07123624]
[0.1031622]
[0.12857097]
[0.16638185]
[0.20813487]
[0.25485321]
[0.26704732]
[0.36944135]
[0.38366552]
[0.41692114]
[0.44394783]
[0.46193203]
[0.46706793]
[0.50766323]
[0.54815417]
[0.56415016]
[0.58057285]
[0.58082662]
[0.58953545]
[0.66091705]
[0.68832773]
[0.69147036]
[0.69545472]
[0.71550836]
[0.71846164]
[0.72851251]
[0.73253501]
[0.75762677]
[0.7591265]
[0.87300511]
[0.90273246]
[0.91013589]
[1.01310855]
[1.04053941]
[1.05419562]
[1.06560493]
[1.07396758]
[1.10528192]
[1.13922944]
[1.1864874]
[1.19045204]
[1.20299504]
[1.23285874]
[1.36716089]
[1.38477451]
[1.39279566]
[1.40378904]
[1.43012508]
[1.44031322]
[1.46778198]
[1.49072245]
[1.50604404]
[1.54217604]
[1.58745621]
[1.58984348]
[1.60561028]
[1.63925687]
[1.64817712]
[1.67300132]
[1.71275035]
[1.80974629]
[1.82496675]
[1.87548857]
[1.87910179]
[1.91043321]
[1.91205376]
[1.92833249]
[1.9665808]
[1.97699844]
[1.97738848]
[2.08373835]
[2.08760397]
[2.09028576]
[2.

In [18]:
def calculate_kernel_matrix(X, gamma='scale'):
    n_samples = X.shape[0]
    kernel_matrix = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(n_samples):
            kernel_matrix[i, j] = rbf_kernel(X[i], X[j], gamma)
    return kernel_matrix


In [25]:
import numpy as np

# Create a vector of size 5 with all elements being 42
vector = np.full(5, 42)

print(vector)


[42 42 42 42 42]


In [210]:
# svr_model = SVR(C=1.0, epsilon=0.1, sigma=1.0, max_iter=100)
# y.flatten()
A,b,P,q,G,h=0,0,0,0,0,0
N=y_train.shape[0]
n_samples, n_features = X.shape
epsilon=0.1
C=1
# e_vec = np.full(N, epsilon)
# C_vec = np.full(N, C)
zero_vec = np.zeros((N,1))
eye_mat = np.eye(N)
y= np.reshape(y, (-1, 1))
def train_svr(X, y, C=1.0, epsilon=0.1, gamma='scale'):
    y= np.reshape(y, (-1, 1))
    global N,P,q,G,h,A,b
    N=y.shape[0]
    n_samples, n_features = X.shape
    epsilon=0.1
    C=1
    # e_vec = np.full(N, epsilon)
    # C_vec = np.full(N, C)
    zero_vec = np.zeros((N,1))
    eye_mat = np.eye(N)
    e_vec = np.full((N,1), epsilon)
    C_vec = np.full((N,1), C)
    zero_mat=np.zeros((N,N))

    # Normalize input features if needed

    # Calculate the kernel matrix
    K = calculate_kernel_matrix(X, gamma)

    # Quadratic programming problem (simplified)
    # P = matrix(np.outer(y.flatten(), y.flatten()) * kernel_matrix)
    # q = matrix(-np.ones(n_samples))
    # G = matrix(np.vstack([-np.eye(n_samples), np.eye(n_samples)]))
    # h = matrix(np.hstack([np.zeros(n_samples), np.ones(n_samples) * C]))
    # P=np.block([[-K, K], [K, -K]])
    
    # q = np.vstack([y-e_vec, -y-e_vec])
    
    # G = np.block([[eye_mat, zero_mat], [zero_mat, -eye_mat]])
    # h = np.vstack([C_vec, zero_vec])
    # A=G
    # b=np.zeros((2*N,1))

    K = calculate_kernel_matrix(X_train, gamma)

    # Construct P
    P = matrix(K)

    # Construct q
    q = matrix(-np.ones((N, 1)))

    # Construct G
    G = matrix(np.vstack([-np.eye(N), np.eye(N)]))

    # Construct h
    # C = 1.0  # Your regularization parameter
    h = matrix(np.vstack([np.zeros((N, 1)), C * np.ones((N, 1))]))

    # q1=matrix(q)
    # print(q1.typecode)
    # print(y.shape," and , ",e_vec.shape)
    # print(G.shape)
    # Solve the optimization problem using cvxopt
    sol = solvers.qp(matrix(P), matrix(q), matrix(G), matrix(h))

    # Extract Lagrange multipliers
    alpha = np.array(sol['x'])
    
    print("optimised at ",alpha.shape)
    # print(alpha)

    # Extract support vectors
    sv_indices = np.where(alpha > 1e-5)[0]
    support_vectors_X = X[sv_indices]
    support_vectors_y = y[sv_indices]
    support_vectors_alpha = alpha[sv_indices]

    # Calculate weights and intercept
    weights = np.sum(support_vectors_alpha[:, None, None] *
                     support_vectors_y[:, :, None] *
                     np.array([rbf_kernel(x, support_vectors_X, gamma) for x in X]), axis=0)

    intercept = np.mean(support_vectors_y - np.sum(weights * np.array([rbf_kernel(x, support_vectors_X, gamma) for x in X]), axis=0))

    return weights, alpha, intercept, {'X': support_vectors_X, 'y': support_vectors_y, 'alpha': support_vectors_alpha}


[ 1  2 -1 -2]
[ 3  4 -3 -4]
[-1 -2  1  2]
[-3 -4  3  4]



In [263]:
def predict_svr(X_test, weights, intercept, support_vectors, alpha,gamma='scale'):
    # for x in X_test:
    #     kernel_values = np.array([rbf_kernel(x, xn) for xn in X_train])
    # print(type(weights))
    # print(type(kernel_values))
    # kernel_values = kernel_values.reshape(1, -1)
    # weights = weights.T
    # print("alpha = ",alpha)
    # print(alpha.shape)
    # kernel_values= np.reshape(kernel_values, (-1, 1))
    # print(kernel_values.shape)
    # Perform the dot product and add the intercept
    # predictions = np.dot(alpha.T, kernel_values) + intercept
    # predictions = np.array(np.dot(alpha.T, np.array([rbf_kernel(x, xn) for xn in X_train])) + intercept for x in X_test)
    
    predictions = []
    for x_test in X_test:
        kernel_values = [rbf_kernel(x_test, xn,gamma) for xn in X_train]
        weighted_sum = sum(alpha_i * kernel_i for alpha_i, kernel_i in zip(alpha, kernel_values))
        prediction = weighted_sum + intercept
        predictions.append(prediction)

    predictions = np.array(predictions)

    
    return predictions

In [21]:
# weights, intercept, support_vectors =train_svr(X_train, y_train, C=1.0, epsilon=0.1, gamma='scale')
# y_pred=predict_svr(X_test,weights,intercept,support_vectors)



In [268]:
def analysis(y_test,y_pred):
    # y_true = y_test  # Replace true_values with the actual values from your dataset
    # y_pred = predict_svr(X_test, weights, intercept, support_vectors, gamma)
    # print(y_pred)
    # print(y_test)
# Calculate the mean squared error (MSE)
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error (MSE): {mse}')

# Calculate the mean absolute error (MAE)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Mean Absolute Error (MAE): {mae}')

# Calculate the R-squared (R2) score
    r2 = r2_score(y_test, y_pred)
    print(f'R-squared (R2) Score: {r2}')

    return r2
    

In [91]:
e_vec = np.full((N,1), 3)
N=y.shape[0]
q.shape
y.shape
e_vec.shape
X.shape
e_vec.shape
y.shape
N
q=np.vstack([y-e_vec, -y-e_vec])

In [121]:
G=np.vstack([eye_mat, -eye_mat])
G.shape
np.linalg.matrix_rank(G)

200

In [136]:
# N=y_train.shape[0]

# j=np.vstack([P,A,G])
j
np.linalg.matrix_rank(j)

320

In [275]:
# weights, alpha,intercept,  support_vectors =train_svr(X_train, y_train, C=1.0, epsilon=0.1, gamma=32)

ma=[0,0,0]
r_b=-100


for C in [1e-4,1e-3,1e-2,1e-1,0.2,0.5,1,2,5,10,20]:
    print("\nnew C ",C,end='\t')
    for epsilon in [1e-4,1e-3,1e-2,1e-1,0.1,0.2,0.5]:
        # print("new epslion ",epsilon,end='\t')
        
            for gamma in [1,2,10,100,1e3,1e4,1e5,1e6]:
            
                with open ("log.txt","a+") as f:
                    # f.write("\n new epsilon : \n")
                
                    weights, alpha, intercept, support_vectors =train_svr(X_train, y_train, C, epsilon, gamma)
                    y_pred=predict_svr(X_test,weights,intercept,support_vectors,alpha,gamma)
                    r2_scorej=analysis(y_test,y_pred)
                    print(r2_scorej)

                    if (r2_scorej>r_b):
                        ma=[C,epsilon,gamma]
                    f.write(f"{C}, {epsilon}, {gamma}, {r2_scorej}\n")
        
            

# # y_pred=y_pred[0]




new C  0.0001	     pcost       dcost       gap    pres   dres
 0: -4.3346e-01 -1.7154e+02  2e+02  3e-17  3e-16
 1: -1.6610e+00 -5.9570e+00  4e+00  2e-16  5e-16
 2: -1.8225e+00 -2.2107e+00  4e-01  2e-16  3e-16
 3: -1.8520e+00 -2.0567e+00  2e-01  2e-16  3e-16
 4: -1.8662e+00 -1.8947e+00  3e-02  2e-16  4e-16
 5: -1.8704e+00 -1.8876e+00  2e-02  2e-16  4e-16
 6: -1.8734e+00 -1.8770e+00  4e-03  1e-16  3e-16
 7: -1.8737e+00 -1.8765e+00  3e-03  1e-16  4e-16
 8: -1.8744e+00 -1.8748e+00  4e-04  2e-16  4e-16
 9: -1.8745e+00 -1.8747e+00  2e-04  2e-16  3e-16
10: -1.8745e+00 -1.8746e+00  4e-05  2e-16  4e-16
11: -1.8745e+00 -1.8746e+00  3e-05  2e-16  3e-16
12: -1.8746e+00 -1.8746e+00  4e-06  2e-16  4e-16
13: -1.8746e+00 -1.8746e+00  2e-06  2e-16  4e-16
Optimal solution found.
optimised at  (160, 1)
[[6.14584473e-05]
 [6.74965465e-07]
 [1.68594930e-06]
 [1.95734176e-07]
 [7.47858315e-06]
 [2.76046907e-07]
 [2.25108336e-07]
 [1.40619666e-07]
 [3.01691291e-05]
 [5.98385440e-05]
 [1.10922835e-05]
 [1.43

KeyboardInterrupt: 

In [55]:
 np.zeros((N,1))

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

array([[4.05568790e-01],
       [3.11934583e-07],
       [6.77429291e-04],
       [1.89064363e-04],
       [5.47291131e-02],
       [2.54582455e-07],
       [1.91398124e-04],
       [1.35054615e-04],
       [3.47351587e-02],
       [2.42426942e-01],
       [5.70235125e-02],
       [6.35493529e-05],
       [2.26833618e-07],
       [1.05915730e-01],
       [1.70135424e-01],
       [9.70711910e-02],
       [4.35713827e-02],
       [5.80801618e-02],
       [1.17057965e-04],
       [1.08157031e-01],
       [1.26134648e-01],
       [6.05864018e-02],
       [3.85851660e-03],
       [4.82562006e-06],
       [7.68570749e-02],
       [5.95362766e-02],
       [1.36908306e-01],
       [1.10368790e-06],
       [2.70979014e-01],
       [2.32232439e-01],
       [2.83942760e-04],
       [7.81460460e-02],
       [2.29704477e-01],
       [1.87920946e-01],
       [5.96735517e-02],
       [5.64435210e-02],
       [1.64994886e-04],
       [4.59424276e-02],
       [1.23245183e-01],
       [7.79369978e-02],


In [229]:
support_vectors['X'].shape

(137, 1)

In [288]:

y_pred=predict_svr(X_test,weights,intercept,support_vectors,alpha,gamma=10000)
analysis(y_test,y_pred)

Mean Squared Error (MSE): 0.554948759175955
Mean Absolute Error (MAE): 0.6700955152140228
R-squared (R2) Score: -0.025503816768542853


-0.025503816768542853

In [249]:
# y_train.shape
# y_pred[1]
y_pred

array([[8.07725919],
       [5.90749682],
       [6.57764444],
       [6.9613407 ],
       [7.84467855],
       [7.9677486 ],
       [7.98001145],
       [6.42546839],
       [6.15214424],
       [7.41196714],
       [7.96539723],
       [5.60994603],
       [6.67865765],
       [8.05495885],
       [5.3238101 ],
       [6.04270196],
       [7.76786482],
       [7.16472236],
       [8.07089446],
       [7.97938901],
       [7.8936038 ],
       [5.92139131],
       [7.38554478],
       [8.0803183 ],
       [7.94317045],
       [7.83871582],
       [8.07347623],
       [7.96689514],
       [7.88113486],
       [7.81130679],
       [5.31545373],
       [6.13503639],
       [7.73829731],
       [8.03474559],
       [7.20468638],
       [8.0385561 ],
       [7.75635194],
       [7.68278359],
       [6.71545373],
       [8.03540076]])

In [250]:
for i in range(40):
    print(y_pred[i],y_test[i])

[8.07725919] 0.46267079162017105
[5.90749682] 0.4456784753225643
[6.57764444] 0.6687565162777134
[6.9613407] -0.8148157213489869
[7.84467855] -0.2221318351602943
[7.9677486] 0.008964390291167464
[7.98001145] 0.942334584183934
[6.42546839] -0.9343121239118178
[6.15214424] -0.9681278464223377
[7.41196714] 0.9331201793944238
[7.96539723] 0.9539393248095179
[5.60994603] -0.9985654016271264
[6.67865765] -0.8883101316814267
[8.05495885] 0.818672412101687
[5.3238101] -0.9990280018447263
[6.04270196] -0.9778415032977705
[7.76786482] 0.9995904676440495
[7.16472236] -0.7421110543987058
[8.07089446] 0.7528132749032542
[7.97938901] 0.942875698810412
[7.8936038] -0.14331916813798545
[5.92139131] 0.45027020246989213
[7.38554478] -0.6357452557317838
[8.0803183] 0.5009964756414914
[7.94317045] 0.9678722193994915
[7.83871582] 0.9976574921094743
[8.07347623] 0.7373059557637529
[7.96689514] 0.9528491363218945
[7.88113486] -0.16468768863977418
[7.81130679] -0.2692043772557255
[5.31545373] 0.25210336190009

In [248]:
analysis(y_test,y_pred)

Mean Squared Error (MSE): 51.846843992077666
Mean Absolute Error (MAE): 7.15648244455533
R-squared (R2) Score: -94.80909141995352


In [32]:
y_pred[4]

array([177.34199982, 177.34199982, 177.34199982, 177.34199982,
       177.34199982, 177.34199982, 177.34199982])

In [31]:
y_pred[1]

array([177.34199982, 177.34199982, 177.34199982, 177.34199982,
       177.34199982, 177.34199982, 177.34199982])

In [30]:
y_test.shape
y_pred.shape
y_pred[0]

array([177.34199982, 177.34199982, 177.34199982, 177.34199982,
       177.34199982, 177.34199982, 177.34199982])

In [ ]:
# import numpy as np
# from sklearn.model_selection import train_test_split

# # Assuming merged_data has columns like 'Date', 'Adj Close', 'Open', 'News_Sentiment', and other relevant features
# features = merged_data[['High', 'Low', 'Volume','Sentiment1','Sentiment2']]  # Adjust features accordingly
# # target = merged_data[['Open','Close']]  # Assuming you have a column 'Next_Day_Adj_Close'
# target = merged_data['Close']

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# def rbf_kernel(X1, X2, gamma='auto'):
#     if gamma == 'auto':
#         # gamma = 1 / len(X1[0])
#         gamma = 1 / X1.shape[0]
#     return np.exp(-gamma * np.linalg.norm(X1 - X2) ** 2)

# def fit_svr(X, y, kernel='rbf', C=1.0, epsilon=0.1, gamma='auto'):
#     n_samples, n_features = X.shape

#     # Compute the kernel matrix
#     kernel_matrix = np.zeros((n_samples, n_samples))
#     for i in range(n_samples):
#         for j in range(n_samples):
#             if kernel == 'rbf':
#                 kernel_matrix[i, j] = rbf_kernel(X[i], X[j], gamma)

#     # Quadratic programming problem to solve the dual problem
#     P = np.outer(y, y) * kernel_matrix
#     q = -np.ones(n_samples)
#     G = np.vstack((-np.eye(n_samples), np.eye(n_samples)))
#     h = np.hstack((np.zeros(n_samples), np.ones(n_samples) * C))

#     # Solve the quadratic programming problem
#     alpha = np.linalg.solve(P, q)

#     # Extract support vectors
#     sv_indices = alpha > 1e-5
#     support_vectors = X[sv_indices]
#     support_vector_labels = y[sv_indices]

#     # Calculate weights and bias
#     weights = np.sum(alpha[sv_indices][:, np.newaxis] * support_vector_labels[:, np.newaxis] * support_vectors, axis=0)
#     bias = np.mean(support_vector_labels - np.dot(support_vectors, weights))

#     return support_vectors, support_vector_labels, weights, bias

# # def predict_svr(X, support_vectors, weights, bias, kernel='rbf', gamma='auto'):
# #     if kernel == 'rbf':
# #         kernel_values = np.array([rbf_kernel(X, sv, gamma) for sv in support_vectors])
# #     else:
# #         kernel_values = np.dot(X, support_vectors.T)

# #     return np.dot(kernel_values, weights) + bias
# def predict_svr(X, support_vectors, weights, bias, kernel='rbf', gamma='auto'):
#     if kernel == 'rbf':
#         kernel_values = np.array([rbf_kernel(X_i, sv, gamma) for X_i in X for sv in support_vectors])
#     else:
#         kernel_values = np.dot(X, support_vectors.T)

#     return np.dot(kernel_values, weights) + bias

# # Usage:
# support_vectors, sv_labels, sv_weights, sv_bias = fit_svr(X_train.values, y_train.values)
# y_pred = predict_svr(X_test.values, support_vectors, sv_weights, sv_bias)

# def analysis(y_test,y_pred):
#     # y_true = y_test  # Replace true_values with the actual values from your dataset
#     # y_pred = predict_svr(X_test, weights, intercept, support_vectors, gamma)

# # Calculate the mean squared error (MSE)
#     mse = mean_squared_error(y_test, y_pred)
#     print(f'Mean Squared Error (MSE): {mse}')

# # Calculate the mean absolute error (MAE)
#     mae = mean_absolute_error(y_test, y_pred)
#     print(f'Mean Absolute Error (MAE): {mae}')

# # Calculate the R-squared (R2) score
#     r2 = r2_score(y_test, y_pred)
#     print(f'R-squared (R2) Score: {r2}')

